In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow.lite as tflite
from tensorflow import keras
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.utils import img_to_array, load_img

Conversion

In [ ]:
model_path = "xception_final_11_0.871.keras"
# model = keras.models.load_model(model_path, custom_objects={"CustomModel": CustomModel})
model = keras.models.load_model(
    model_path,
)

In [ ]:
converter = tflite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open("clothing-model.tflite", "wb") as f_out:
    f_out.write(tflite_model)

Using the litert model

In [ ]:
# Preprocess a single image
def preprocess_image(image_path, input_size=299):
    """Preprocess an image for prediction."""
    img = load_img(image_path, target_size=(input_size, input_size))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = preprocess_input(img_array)  # Preprocessing specific to Xception
    return img_array

In [ ]:
test_image_path = (
    "../data/clothing-dataset-small/test/shoes/0dd87e47-ca85-4d5c-9fd1-59f5a01eb656.jpg"
)
input_size = 299
class_names = [
    "dress",
    "hat",
    "longsleeve",
    "outwear",
    "pants",
    "shirt",
    "shoes",
    "shorts",
    "skirt",
    "t-shirt",
]

In [ ]:
interpreter = tflite.Interpreter(model_path="clothing-model.tflite")
interpreter.allocate_tensors()
input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

In [ ]:
preprocessed_image = preprocess_image(test_image_path, input_size)
interpreter.set_tensor(input_index, preprocessed_image)
interpreter.invoke()

In [ ]:
preds = interpreter.get_tensor(output_index)
predicted_class = class_names[np.argmax(preds[0])]
scores = dict(zip(class_names, preds[0]))
print(f"Predicted class: {predicted_class}\nscores: {scores}")